In [1]:
import os
import pickle
import numpy as np
import cv2
import cvzone
import numpy as np
from datetime import datetime
import face_recognition

In [2]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import storage

In [3]:
# Check if the Firebase app is already initialized
if not firebase_admin._apps:
    cred = credentials.Certificate("serviceAccountKey.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': "https://faceattendanceproject-45968-default-rtdb.firebaseio.com/",
        'storageBucket': "faceattendanceproject-45968.appspot.com"
    })
else:
# If the app is already initialized, get the existing app
    app = firebase_admin.get_app()
bucket = storage.bucket()

In [4]:
# Importing the mode images into a list
folderModePath = r'Resources\Modes'
modePathList = os.listdir(folderModePath)
imgModeList = []
for path in modePathList:
    imgModeList.append(cv2.imread(os.path.join(folderModePath, path)))
print(len(imgModeList))

4


In [5]:
# Loading the encoding file
print("Loading Encode File ...")
file = open('EncodeFile.p', 'rb')
encodeListKnownWithIds = pickle.load(file)
file.close()
encodeListKnown, studentIds = encodeListKnownWithIds
print(studentIds)
print("Encode File Loaded")

Loading Encode File ...
['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9']
Encode File Loaded


In [6]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
imgBackgroundPath = r'Resources\background.png'  # Use a raw string 
imgBackground = np.zeros((480, 640, 3), dtype=np.uint8)
imgBackground = cv2.imread(imgBackgroundPath)

modeType = 0
counter = 0
id = -1
imgStudent = []

while True:
    success, img = cap.read()
    if not success:
        print("Error reading frame from the video source")
        break  
    
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    faceCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)

    imgBackground[162:162 + 480, 55:55 + 640] = img
    imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]

    if faceCurFrame:
        for encodeFace, faceLoc in zip(encodeCurFrame, faceCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                bbox = 55 + x1, 162 + y1, x2 - x1, y2 - y1
                imgBackground = cvzone.cornerRect(imgBackground, bbox, rt=0)
                id = studentIds[matchIndex]
                if counter == 0:
                    cvzone.putTextRect(imgBackground, "Loading", (275, 400))
                    cv2.imshow("Face Attendance", imgBackground)
                    cv2.waitKey(1)
                    counter = 1
                    modeType = 1

        if counter != 0:

            if counter == 1:
                # Get the Data
                studentInfo = db.reference(f'Students/{id}').get()
                print(studentInfo)
                # Get the Image from the storage
                blob = bucket.get_blob(f'Images/{id}.png')
                array = np.frombuffer(blob.download_as_string(), np.uint8)
                imgStudent = cv2.imdecode(array, cv2.COLOR_BGRA2BGR)
                # Update data of attendance
                datetimeObject = datetime.strptime(studentInfo['last_attendance_time'],
                                                   "%Y-%m-%d %H:%M:%S")
                secondsElapsed = (datetime.now() - datetimeObject).total_seconds()
                print(secondsElapsed)

                if secondsElapsed > 60: # Time after which a particular student can mark again
                    ref = db.reference(f'Students/{id}')
                    studentInfo['total_attendance'] += 1
                    ref.child('total_attendance').set(studentInfo['total_attendance'])
                    ref.child('last_attendance_time').set(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                else:
                    modeType = 3
                    counter = 0
                    imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]

            if modeType != 3:

                if 10 < counter < 20:
                    modeType = 2

                imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]

                if counter <= 10:
                    cv2.putText(imgBackground, str(studentInfo['total_attendance']), (861, 125),
                                cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)
                    cv2.putText(imgBackground, str(studentInfo['major']), (1006, 550),
                                cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    cv2.putText(imgBackground, str(id), (1006, 493),
                                cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    cv2.putText(imgBackground, str(studentInfo['standing']), (910, 625),
                                cv2.FONT_HERSHEY_COMPLEX, 0.6, (100, 100, 100), 1)
                    cv2.putText(imgBackground, str(studentInfo['year']), (1025, 625),
                                cv2.FONT_HERSHEY_COMPLEX, 0.6, (100, 100, 100), 1)
                    cv2.putText(imgBackground, str(studentInfo['starting_year']), (1125, 625),
                                cv2.FONT_HERSHEY_COMPLEX, 0.6, (100, 100, 100), 1)

                    (w, h), _ = cv2.getTextSize(studentInfo['name'], cv2.FONT_HERSHEY_COMPLEX, 1, 1)
                    offset = (414 - w) // 2
                    cv2.putText(imgBackground, str(studentInfo['name']), (808 + offset, 445),
                                cv2.FONT_HERSHEY_COMPLEX, 1, (50, 50, 50), 1)

                    imgBackground[175:175 + 216, 909:909 + 216] = imgStudent

                counter += 1

                if counter >= 20:   # Reseting attendance screen 
                    counter = 0
                    modeType = 0
                    studentInfo = []
                    imgStudent = []
                    imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
    else:
        modeType = 0
        counter = 0
    cv2.imshow("Face Attendance", imgBackground)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Close the camera if 'q' is pressed
        break

{'last_attendance_time': '2024-01-23 16:33:27', 'major': 'Physics', 'name': 'Elon Musk', 'standing': 'G', 'starting_year': 2021, 'total_attendance': 8, 'year': 3}
1830.138866
{'last_attendance_time': '2024-01-23 17:03:57', 'major': 'Physics', 'name': 'Elon Musk', 'standing': 'G', 'starting_year': 2021, 'total_attendance': 9, 'year': 3}
5.20553
{'last_attendance_time': '2024-01-23 17:03:57', 'major': 'Physics', 'name': 'Elon Musk', 'standing': 'G', 'starting_year': 2021, 'total_attendance': 9, 'year': 3}
7.714197
{'last_attendance_time': '2024-01-23 17:03:57', 'major': 'Physics', 'name': 'Elon Musk', 'standing': 'G', 'starting_year': 2021, 'total_attendance': 9, 'year': 3}
14.838937
{'last_attendance_time': '2022-12-11 00:54:34', 'major': 'Singing', 'name': 'Olivia Rodrigo', 'standing': 'B', 'starting_year': 2022, 'total_attendance': 4, 'year': 2}
35309413.53511
{'last_attendance_time': '2024-01-23 17:04:47', 'major': 'Singing', 'name': 'Olivia Rodrigo', 'standing': 'B', 'starting_year'

KeyboardInterrupt: 

: 